In [7]:
from pipeline.util.platform import start_spark, start_spark_local
from pipeline.util.storage import read_csv, read_parquet, write_parquet, write_repart_parquet, write_delta, write_deltatable
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [8]:
spark, loggrer, conf = start_spark(app_name='addessmatch_etl_job')

local


In [29]:
df_bld = read_parquet(spark, "{storage_curated}/data/in/addressmatchres".format(storage_curated = spark.conf.get('storage.curated')))

In [30]:
df_bld_id = df_bld.withColumn("BUILDING_ID", F.monotonically_increasing_id() + F.unix_timestamp(F.current_timestamp()))

In [31]:
df_bld_id.createOrReplaceTempView("building_in")

In [21]:
loc = "'{storage_curated}/data/out/building'".format(storage_curated = spark.conf.get('storage.curated'))

#spark.sql(f"DROP TABLE IF EXISTS BUILDING")
spark.sql(f"CREATE OR REPLACE TABLE BUILDING ( \
          ID INT, \
          BUILDINGADDRESS STRING, \
          UPRN VARCHAR(12), \
          BUILDING_ID BIGINT ) \
          USING DELTA  \
          LOCATION {loc}")

DataFrame[]

In [33]:
spark.sql(f"MERGE INTO BUILDING \
          USING building_in \
          ON BUILDING.ID = building_in.id \
          WHEN MATCHED THEN \
            UPDATE SET \
              BUILDINGADDRESS = building_in.buildingaddress, \
              UPRN = building_in.UPRN \
          WHEN NOT MATCHED THEN \
            INSERT ( \
              ID, BUILDINGADDRESS, UPRN, BUILDING_ID \
            ) VALUES ( \
              building_in.id, building_in.buildingaddress, building_in.UPRN, building_in.BUILDING_ID \
            )")

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [38]:
#df_admatchres.printSchema()
#spark.sql("describe table building_in").show()
#spark.sql("select * from BUILDING").show()
#spark.sql("select * from BUILDING a join ADDRESS b on a.UPRN=b.UPRN").show(truncate=False)
#spark.sql("show tables").show()
#spark.sql("drop table addressmatchres")


+---+--------------------------------------------------------------------------+------------+-----------+------------+-------+--------+---------------------------------------------------------------+-----------+
|ID |BUILDINGADDRESS                                                           |UPRN        |BUILDING_ID|UPRN        |OUTCODE|LOCALITY|ADDRESS                                                        |ADDRESS_ID |
+---+--------------------------------------------------------------------------+------------+-----------+------------+-------+--------+---------------------------------------------------------------+-----------+
|9  |36 Southernhay E, Exeter EX1 1NX                                          |100041046158|1710785012 |100041046158|1NX    |null    |36 SOUTHERNHAY EAST EXETER DEVON EX1 1NX                       |1710731536 |
|4  |FLAT 52D MORTIMER HOUSE GRENDON ROAD EX1 2NL                              |100040215247|1710785008 |100040215247|2NL    |null    |FLAT 52D MORTIMER